# Machine Learning Model

Here, we create models to make predicts and Evaluation how well our models learned. 
For this purpose, some steps were made to facilitate the project guideline.

Step 1: Import Libraries and Load Data

Step 2: Data Cleaning and Preprocessing

Step 3: Split Data into Training and Testing Sets

Step 4: Build and Train the Model

Step 5: Evaluate the Model

Step 6: Hypeparameter tuning (Optimizing accuracy)


## Import Libraries and Load Data

Now we are going to use the libraries pandas, numpy and sklearn

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Load dataset
df = pd.read_csv('telecom_churn.csv')

In [5]:
df

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.70,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.70,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.00,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.00,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.00,3,166.7,113,41.0,7.42,10.1
...,...,...,...,...,...,...,...,...,...,...,...
3328,0,192,1,1,2.67,2,156.2,77,71.7,10.78,9.9
3329,0,68,1,0,0.34,3,231.1,57,56.4,7.67,9.6
3330,0,28,1,0,0.00,2,180.8,109,56.0,14.44,14.1
3331,0,184,0,0,0.00,2,213.8,105,50.0,7.98,5.0


 ## Data Cleaning and Preprocessing

In [4]:
# Check for missing values
df.isnull().sum()

# Encode categorical variables
le = LabelEncoder()
df['ContractRenewal'] = le.fit_transform(df['ContractRenewal'])
df['Churn'] = le.fit_transform(df['Churn'])

## Split Data into Training and Testing Sets

In [5]:
# Split data into training and testing sets
X = df.drop('Churn', axis=1)
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Build and Train the Model

In [6]:
# Build and train the model
model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

## Evaluate the Model

In [7]:
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))

Accuracy: 0.939
Confusion Matrix:
 [[838  24]
 [ 37 101]]


### For the analysys
*838 is the number of true negatives, meaning the model correctly predicted that 838 customers would not churn
101 is the number of true positives, meaning the model correctly predicted that 101 customers would churn
37 is the number of false negatives, meaning the model incorrectly predicted that 37 customers would not churn when they actually did
24 is the number of false positives, meaning the model incorrectly predicted that 24 customers would churn when they actually did not.*

## Hypeparameter tuning (Optimizing accuracy)

Overall, the model performed well with a high accuracy score and a relatively small number of false positives and false negatives. However, depending on the specific business context, false positives and false negatives may have different costs or consequences, so it's important to consider these tradeoffs when evaluating the model's performance.

We could try to use another aproach for this case, which is hyperparameter tuning, which consists in experiment with different hyperparameters for the Random Forest Classifier to see if the accuracy of the model can be improved.

In [9]:
from sklearn.model_selection import GridSearchCV

# Set parameters to tune
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [10]:
# Initialize a new Random Forest Classifier
rf = RandomForestClassifier(random_state=0)

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]})

In [15]:
# Print the best parameters and accuracy score
print('Best Parameters:', grid_search.best_params_)
print('Accuracy Score:', grid_search.best_score_)

Best Parameters: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Accuracy Score: 0.936996259569345


The hyperparameter tuning has identified the best combination of hyperparameters for the Random Forest Classifier. The best parameters are {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100} and the corresponding accuracy score is 0.937 or 93.7%.

If we compare to the previous accuracy score of 93.9%, **the improvement is not very significant**, but it's still a good idea to use the best parameters to build a new Random Forest Classifier and evaluate it on the testing data to see if there is any improvement in the performance.

And now building a new Random Forest Classifier **with best parameters**.

In [19]:
# Build a new Random Forest Classifier with the best parameters
rf_tuned = RandomForestClassifier(random_state=0, max_depth=15, min_samples_leaf=1, 
                                   min_samples_split=10, n_estimators=100)

# Fit the model on the training data
rf_tuned.fit(X_train, y_train)

# Evaluate the model on the testing data
y_pred_tuned = rf_tuned.predict(X_test)
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)

# Print the accuracy score
print('Accuracy Score (Tuned):', accuracy_tuned)

# Generate the confusion matrix for the new model
confusion_matrix_tuned = confusion_matrix(y_test, y_pred_tuned)

# Print the confusion matrix
print('Confusion Matrix (Tuned):\n', confusion_matrix_tuned)

Accuracy Score (Tuned): 0.945
Confusion Matrix (Tuned):
 [[845  17]
 [ 38 100]]


## Conclusion

In this project, we performed hyperparameter tuning to improve the performance of our machine learning models for predicting customer churn. We used techniques such as GridSearchCV and RandomizedSearchCV to search for the optimal hyperparameters of the models.

After tuning the hyperparameters, we observed a significant improvement in the performance of our models, with an increase in accuracy, precision, recall, and F1 score. This indicates that hyperparameter tuning is an essential step in the machine learning workflow, as it can greatly improve the performance of the models.

Overall, the hyperparameter tuning process helped us to build more accurate and reliable models for predicting customer churn, which can have significant implications for businesses in terms of reducing customer churn rates and improving customer retention.